In [1]:
import sys
import os

sys.path.append("/home/laststar/source/OpenLLM-Rec/source")

In [2]:
import easydict

# jupyter didn't support argparse. so, I use 'easydict' module
args = easydict.EasyDict({
    ################
    # Dataset
    ################
    'dataset_code': 'ml-100k', # ml-100k, beauty, games
    'min_rating': 0,  # default: 0
    'min_uc': 5,  # default: 5
    'min_sc': 5,  # default: 5
    'seed': 42,  # default: 42

    ################
    # Dataloader
    ################
    'train_batch_size': 64,  # default: 64
    'val_batch_size': 64,  # default: 64
    'test_batch_size': 64,  # default: 64
    'num_workers': 0,  # default: 8
    'sliding_window_size': 1.0,  # default: 1.0
    'negative_sample_size': 10,  # default: 10

    ################
    # Trainer
    ################
    # optimization #
    'device': 'cuda',  # default: 'cuda'  # choices: ['cpu', 'cuda']
    'num_epochs': 500,  # default: 500
    'optimizer': 'AdamW',  # default: 'AdamW'  # choices: ['AdamW', 'Adam']
    'weight_decay': 0.01,  # default: None
    'adam_epsilon': 1e-9,  # default: 1e-9
    'momentum': None,  # default: None
    'lr': 0.001,  # default: 0.001
    'max_grad_norm': 5.0,  # default: 5.0
    'enable_lr_schedule': True,  # default: True
    'decay_step': 10000,  # default: 10000
    'gamma': 1,  # default: 1
    'enable_lr_warmup': True,  # default: True
    'warmup_steps': 100,  # default: 100

    # evaluation #
    'val_strategy': 'iteration',  # default: 'iteration'  # choices: ['epoch', 'iteration']
    'val_iterations': 500,  # default: 500  # only for iteration val_strategy
    'early_stopping': True,  # default: True
    'early_stopping_patience': 20,  # default: 20
    'metric_ks': [1, 5, 10, 20, 50],  # default: [1, 5, 10, 20, 50]
    'rerank_metric_ks': [1, 5, 10],  # default: [1, 5, 10]
    'best_metric': 'Recall@10',  # default: 'Recall@10'
    'rerank_best_metric': 'NDCG@10',  # default: 'NDCG@10'
    'use_wandb': False,  # default: False

    ################
    # Retriever Model
    ################
    'model_code': 'bert',  # default: None
    'bert_max_len': 100,  # default: 50
    'bert_hidden_units': 256,  # default: 64
    'bert_num_blocks': 2,  # default: 2
    'bert_num_heads': 4,  # default: 2
    'bert_head_size': 32,  # default: 32
    'bert_dropout': 0.1,  # default: 0.2
    'bert_mask_prob': 0.15,  # default: 0.25
    
    # bertrec
    'train_negative_sampler_code': 'random',
    'train_negative_sample_size': 0,
    'train_negative_sampling_seed': 0,
    'test_negative_sampler_code': 'random',
    'test_negative_sample_size': 100,
    'test_negative_sampling_seed': 98765,
    'model_init_seed': 0,
    'num_gpu': 1,
    'optimizer': 'Adam',
    'log_period_as_iter': 12800,
    
    ################
    # LLM Model
    ################
    'llm_base_model': 'meta-llama/Llama-2-7b-hf',  # default: 'meta-llama/Llama-2-7b-hf'
    'llm_base_tokenizer': 'meta-llama/Llama-2-7b-hf',  # default: 'meta-llama/Llama-2-7b-hf'
    'llm_max_title_len': 32,  # default: 32
    'llm_max_text_len': 1536,  # default: 1536
    'llm_max_history': 20,  # default: 20
    'llm_train_on_inputs': False,  # default: False
    'llm_negative_sample_size': 19,  # default: 19  # 19 negative & 1 positive
    'llm_system_template': "Given user history in chronological order, recommend an item from the candidate pool with its index letter.",  # default: "Given user history in chronological order, recommend an item from the candidate pool with its index letter."
    'llm_input_template': 'User history: {}; \n Candidate pool: {}',  # default: 'User history: {}; \n Candidate pool: {}'
    'llm_load_in_4bit': True,  # default: True
    'llm_retrieved_path': "/home/laststar/data/model/OpenLLM-Rec",  # default: None
    'llm_cache_dir': None,  # default: None

    ################
    # Lora
    ################
    'lora_r': 8,  # default: 8
    'lora_alpha': 32,  # default: 32
    'lora_dropout': 0.05,  # default: 0.05
    'lora_target_modules': ['q_proj', 'v_proj'],  # default: ['q_proj', 'v_proj']
    'lora_num_epochs': 1,  # default: 1
    'lora_val_iterations': 100,  # default: 100
    'lora_early_stopping_patience': 20,  # default: 20
    'lora_lr': 1e-4,  # default: 1e-4
    'lora_micro_batch_size': 16,  # default: 16

    #################
    # Custom
    #################
    'alpaca_file': "../source/data/dataloader/templates"
})

In [3]:
EXPORT_ROOT = "/home/laststar/data/model/OpenLLM-Rec/bert"

In [4]:
import data.datasets
import data.dataloader
from data.dataloader import *
from data.datasets import *

In [5]:
train, val, test = dataloader_factory(args)

Already preprocessed. Skip preprocessing
Negatives samples exist. Loading.
Negatives samples exist. Loading.


In [6]:
len(train.dataset)

610

In [7]:
import os
import torch

os.environ['TOKENIZERS_PARALLELISM'] = 'false'

import wandb
import argparse

from model import *
from data.dataloader import*
from trainer import *

In [8]:
model = BERT4Rec(args)

In [9]:
trainer = BERTTrainer(args, model, train, val, test, EXPORT_ROOT)

In [10]:
from datetime import datetime
print("current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

current time: 2024-11-29 10:02:59


In [11]:
trainer.train()

  0%|          | 0/10 [00:00<?, ?it/s]

Metric MRR@1 not found...
Metric MRR@5 not found...
Metric MRR@10 not found...
Metric MRR@20 not found...
Metric MRR@50 not found...
Update Best Recall@10 Model at 1


Training Progress:   0%|          | 0/500 [00:00<?, ?epoch/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

...

  0%|          | 0/10 [00:00<?, ?it/s]

Metric MRR@1 not found...
Metric MRR@5 not found...
Metric MRR@10 not found...
Metric MRR@20 not found...
Metric MRR@50 not found...


  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

Metric MRR@1 not found...
Metric MRR@5 not found...
Metric MRR@10 not found...
Metric MRR@20 not found...
Metric MRR@50 not found...
Metric epoch not found...
Metric loss not found...


In [12]:
from datetime import datetime
print("current time:", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))

current time: 2024-11-29 10:09:44


In [13]:
trainer.test()

Test best model with test set!


/home/laststar/source/OpenLLM-Rec/source/trainer/bert.py:173: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  best_model = torch.load(os.path.join(self.export_root, 'models', 

  0%|          | 0/10 [00:00<?, ?it/s]

{'Recall@50': 0.7162683844566345, 'NDCG@50': 0.7162683844566345, 'Recall@20': 0.7162683844566345, 'NDCG@20': 0.7162683844566345, 'Recall@10': 0.7162683844566345, 'NDCG@10': 0.7162683844566345, 'Recall@5': 0.7162683844566345, 'NDCG@5': 0.7162683844566345, 'Recall@1': 0.7162683844566345, 'NDCG@1': 0.7162683844566345}
